In [31]:
from collections import Counter
from wordcloud import WordCloud
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt  # 형태소 분석기

In [32]:
client = MongoClient('mongodb://192.168.0.50')
db_name = client['DB_SGMN']
collection = db_name['COL_SCRAPPING_TOSS_COMMENT_HISTORY']

In [33]:
# 데이터 로드

find_data = collection.find()
toss_data = list(find_data)

In [34]:
df_toss_data = pd.DataFrame(toss_data)
df_toss_data.head(3)

,_id,COMMENT,CREATED_AT,DATE,DATETIME,SYMBOL,UPDATED_AT
0,675082d6239b5012b3c84a0b,머선일이야,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:36:32+09:00,AADI,2024-11-21 23:58:33
1,675082d6239b5012b3c84a0c,뭐냐 너..왜 희망 갖게하냐,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:07:42+09:00,AADI,2024-11-21 23:58:34
2,675082d6239b5012b3c84a0d,몇개월 기다리다가 조금 올라서 뺏더니 잘 올라가네.. ㅠ,2024-11-21 23:59:13.068,2024-11-22,2024-11-22T01:00:04+09:00,AADI,2024-11-21 23:58:34


In [35]:
df_toss_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251836 entries, 0 to 251835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   _id         251836 non-null  object        
 1   COMMENT     251836 non-null  object        
 2   CREATED_AT  251836 non-null  datetime64[ns]
 3   DATE        251836 non-null  object        
 4   DATETIME    251836 non-null  object        
 5   SYMBOL      251836 non-null  object        
 6   UPDATED_AT  251836 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 13.4+ MB


In [36]:
# 3. CREATED_AT 필드 날짜 변환

# CREATED_AT을 datetime 형식으로 변환
df_toss_data['CREATED_AT'] = pd.to_datetime(df_toss_data['CREATED_AT'])

# 주 단위로 그룹화 준비 (연도-주 형식 추가)
df_toss_data['YEAR_WEEK'] = df_toss_data['CREATED_AT'].dt.strftime('%Y-%U')

In [37]:
it_sentence = df_toss_data['COMMENT'].tolist() # array list 로 변환 05번 it_sentence 변환 참조

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(stop_words = ['믿음', '올라간다', '진입']
                , ngram_range = (1,2))
tfidfVectorizer.fit(it_sentence)

TfidfVectorizer(ngram_range=(1, 2), stop_words=['믿음', '올라간다', '진입'])

In [39]:
tfidfVectorizer.vocabulary_

{'머선일이야': 168921,
 '뭐냐': 182899,
 '희망': 448967,
 '갖게하냐': 58407,
 '뭐냐 희망': 182925,
 '희망 갖게하냐': 448968,
 '몇개월': 171837,
 '기다리다가': 92701,
 '조금': 359686,
 '올라서': 303009,
 '뺏더니': 216882,
 '올라가네': 301998,
 '몇개월 기다리다가': 171838,
 '기다리다가 조금': 92715,
 '조금 올라서': 359866,
 '올라서 뺏더니': 303067,
 '뺏더니 올라가네': 216884,
 '얘는': 274514,
 '얼마까지': 280459,
 '갈까요': 55932,
 '얘는 얼마까지': 274620,
 '얼마까지 갈까요': 280467,
 '애플': 271354,
 '2026년': 11276,
 '출시': 396246,
 '목표로': 175694,
 '시리': 248385,
 '전면': 352563,
 '개편': 62399,
 '추진': 395578,
 '주요내용': 371378,
 '대규모': 129562,
 '언어모델': 279277,
 'llm': 39622,
 '기반의': 93970,
 '새로운': 229570,
 '개발': 61029,
 '중으로': 375105,
 '프로젝트': 423547,
 '진행': 383911,
 '2025년': 10995,
 'wwdc에서': 45950,
 'ios': 38898,
 '19와': 8024,
 'macos': 39832,
 '16의': 7000,
 '일부로': 333369,
 '공개': 74635,
 '예정': 293206,
 '정식': 356539,
 '자연스러운': 341923,
 '대화': 132403,
 '복잡한': 204383,
 '작업': 342730,
 '처리': 389577,
 '타사': 402372,
 '연동': 289612,
 '기능': 92257,
 '강화': 58277,
 '요약': 309166,
 '현재': 440305,
 '별도': 19

In [40]:
tfidfVectorizer.idf_, tfidfVectorizer.get_feature_names_out()
tfidfVectorizer.idf_.shape, tfidfVectorizer.get_feature_names_out().shape


((449929,), (449929,))

In [41]:
result_vectors = tfidfVectorizer.transform(it_sentence) # 서비스용
result_vectors.toarray().shape, tfidfVectorizer.get_feature_names_out().shape

((251836, 449929), (449929,))

In [42]:
## 문장간 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity
df_similarity = cosine_similarity(result_vectors[:10], result_vectors[:10])
df_similarity

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.00924653, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [43]:
# 결과 보기 쉽게 변환
pd.DataFrame(data = df_similarity
             , index = it_sentence[:10]
             , columns = it_sentence[:10])

,머선일이야,뭐냐 너..왜 희망 갖게하냐,몇개월 기다리다가 조금 올라서 뺏더니 잘 올라가네.. ㅠ,얘는 얼마까지 갈까요,"애플, 2026년 출시 목표로 시리 전면 개편 추진\n\n(주요내용)\n1. 대규모 언어모델(LLM) 기반의 새로운 시리 개발 중으로 'LLM 시리' 프로젝트 진행\n2. 2025년 WWDC에서 iOS 19와 macOS 16의 일부로 공개 예정, 2026년 봄 정식 출시\n3. 자연스러운 대화, 복잡한 작업 처리, 타사 앱 연동 기능 강화\n\n(요약)\n- 현재 별도 앱으로 테스트 중이며 기존 시리를 완전히 대체할 예정\n- ChatGPT 수준의 대화형 인터페이스와 인공지능 기능 구현\n- 텍스트 생성, 요약, 앱 제어 등 다양한 기능 통합\n- iOS 18에서 단계적으로 AI 기능 도입하며 새로운 시리 출시 준비\n- 사용자 개인정보 보호에 중점을 둔 차별화 전략 추구\n\n(핵심요약)\n애플이 ChatGPT에 대응하기 위해 LLM 기반의 차세대 시리를 개발하여 2026년 봄 출시 예정",거 횡보가 너무 심한거 아니요,흠...,히히 나도 애플 주주 ~,힘내라 짝 힘내라 짝,최고가 가겠는데
머선일이야,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
뭐냐 너..왜 희망 갖게하냐,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
몇개월 기다리다가 조금 올라서 뺏더니 잘 올라가네.. ㅠ,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
얘는 얼마까지 갈까요,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0
"애플, 2026년 출시 목표로 시리 전면 개편 추진\n\n(주요내용)\n1. 대규모 언어모델(LLM) 기반의 새로운 시리 개발 중으로 'LLM 시리' 프로젝트 진행\n2. 2025년 WWDC에서 iOS 19와 macOS 16의 일부로 공개 예정, 2026년 봄 정식 출시\n3. 자연스러운 대화, 복잡한 작업 처리, 타사 앱 연동 기능 강화\n\n(요약)\n- 현재 별도 앱으로 테스트 중이며 기존 시리를 완전히 대체할 예정\n- ChatGPT 수준의 대화형 인터페이스와 인공지능 기능 구현\n- 텍스트 생성, 요약, 앱 제어 등 다양한 기능 통합\n- iOS 18에서 단계적으로 AI 기능 도입하며 새로운 시리 출시 준비\n- 사용자 개인정보 보호에 중점을 둔 차별화 전략 추구\n\n(핵심요약)\n애플이 ChatGPT에 대응하기 위해 LLM 기반의 차세대 시리를 개발하여 2026년 봄 출시 예정",0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.009247,0.0,0.0
거 횡보가 너무 심한거 아니요,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0
흠...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
히히 나도 애플 주주 ~,0.0,0.0,0.0,0.0,0.009247,0.0,0.0,1.000000,0.0,0.0
힘내라 짝 힘내라 짝,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0
최고가 가겠는데,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0


In [49]:
new_sentence = '파이썬은 데이터 과학, 웹 개발, 자동화 스크립트 작성 등 다양한 프로그래밍 작업에 널리 사용되는 언어이다.'
new_tfidfVectorizer = tfidfVectorizer.transform([new_sentence])
new_tfidfVectorizer

<1x449929 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [50]:
similarity_array = cosine_similarity(new_tfidfVectorizer, result_vectors)
similarity_array

array([[0., 0., 0., ..., 0., 0., 0.]])

In [51]:
argmax_similarity = similarity_array.argmax()
it_sentence[argmax_similarity]

'금요일은 과학'